Ανοίγω το αρχείο κάνω mapping και ταυτόχρονα γεμίζω τα διανύσματα που θα χρησιμοποιήσω για να φτιάξω τον sparse πίνακα

In [1]:
ratingsFile = open('pruned_data.csv',encoding = 'utf8',errors='ignore')
users = {}
business = {}
user_counter = 0
business_counter = 0
row = []
col = []
data = []
for line in ratingsFile:
    lineValues = line.split(',')
    user_id = lineValues[0]
    business_id = lineValues[1]
    rating = float(lineValues[2])
    data.append(rating)
    if user_id not in users:
        row.append(user_counter)
        users[user_id] = user_counter
        user_counter += 1
    else:
        row_number = users[user_id]
        row.append(row_number)
    if business_id not in business:
        col.append(business_counter)
        business[business_id] = business_counter
        business_counter += 1
    else:
        col_number = business[business_id]
        col.append(col_number)
ratingsFile.close()

Δημιουργώ τον sparse πίνακα με βάση τα row col και data διανύσματα που έβγαλα από το αρχείο

In [3]:
import scipy.sparse as sp_sparse
import numpy as np
row = np.array(row)
col = np.array(col)
data = np.array(data)
M = sp_sparse.csr_matrix((data,(row,col)),shape=(len(users),len(business)))

[   0    1    2 ...  767 1189 3023]
[   0    0    0 ... 4793 4793 4793]
  (0, 0)	4.0
  (0, 1)	4.0
  (0, 2)	5.0
  (0, 3)	4.0
  (0, 4)	4.0
  (0, 5)	5.0
  (0, 6)	4.0
  (0, 7)	4.0
  (0, 8)	2.0
  (0, 9)	5.0
  (0, 10)	5.0
  (0, 11)	4.0
  (0, 12)	5.0
  (0, 13)	4.0
  (0, 14)	4.0
  (0, 15)	4.0
  (0, 16)	2.0
  (0, 17)	3.0
  (0, 18)	4.0
  (0, 19)	4.0
  (0, 20)	5.0
  (0, 21)	3.0
  (0, 22)	4.0
  (0, 23)	4.0
  (0, 24)	4.0
  :	:
  (4792, 2322)	4.0
  (4792, 2423)	5.0
  (4792, 2741)	4.0
  (4792, 2845)	2.0
  (4792, 2982)	5.0
  (4792, 3138)	3.0
  (4793, 40)	5.0
  (4793, 63)	5.0
  (4793, 414)	3.0
  (4793, 767)	5.0
  (4793, 814)	4.0
  (4793, 916)	3.0
  (4793, 972)	4.0
  (4793, 1076)	5.0
  (4793, 1189)	1.0
  (4793, 1634)	3.0
  (4793, 1928)	2.0
  (4793, 2218)	5.0
  (4793, 2388)	4.0
  (4793, 2593)	5.0
  (4793, 2995)	3.0
  (4793, 3023)	5.0
  (4793, 3068)	5.0
  (4793, 3429)	5.0
  (4793, 3893)	3.0
[[4. 4. 5. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. 

Διαλέγω τυχαία 5% του συνόλου των μη μηδενικών μετρήσεων και αφαιρώ μη μηδενικά κελιά του πίνακα μηδενίζοντας τα

In [ ]:
from scipy.sparse import csr_matrix
import random
non_zero_cells = csr_matrix.nonzero(M)
removed_cells_sp = sp_sparse.lil_matrix((len(users),len(business)),dtype=float)
num_non_zero = len(non_zero_cells[0])
cells_to_remove = int(0.05*len(non_zero_cells[0]))
while cells_to_remove > 0:
    k = random.randint(0,num_non_zero-1)
    i = non_zero_cells[0][k]
    j = non_zero_cells[1][k]
    if (i,j) not in removed_cells:
        removed_cells_sp[i,j] = M[i,j]
        M[i,j] = 0
        cells_to_remove -= 1
print(removed_cells_sp)

In [ ]:
print(len(removed_cells))

In [6]:
import sklearn.metrics as metrics
M_cosine_similarity = metrics.pairwise.cosine_similarity(M)
#M_cosine_similarity = csr_matrix(M_cosine_similarity)
print(M_cosine_similarity.shape)

(4794, 4794)


* Εδώ θέτω αρχικά την διαγώνιο του πίνακα με τα similarities με 0 ώστε να μην επιρεάζει στην εντοπισμό των K πιο όμοιων χρηστών.
* Για κάθε κελί που έχω διαγράψει τη βαθμολογία βάση της θέσης του στον πίνακα κάνω πρόβλεψη.
* Κοιτάω σε ποιον χρήστη αντιστοιχεί η βαθμολογία.
* Για τον χρήστη αυτό παίρνω την αντίστοιχη γραμμή του στον πίνακα με τα similarities με τους άλλου χρήστες.
* Κοιτάω να δω σε ποιά επιχειρήση-column ανοίκει η βαθμολογία.
* Για την επιχειρήση αυτή παίρνω όλη τη στήλη που της αντιστοιχεί(j) από τον πίνακα reviews(M) και κρατάω τις θέσεις των χρηστών που έχουν δώσει μη μηδενική βαθμολογία.
* Τραβάω το index από την γραμμή με τα similarities του πιο όμοιου χρήστη (argmax).
* Αν ανοίκει στη λίστα με τους χρήστες που έχουν δώσει μη μηδενική βαθμολογία προστίθεται στο σύνολο με τους Κ πιο όμοιους χρήστες.
* Αλλιώς το similarity του είναι αδιάφορο το μηδενίζουμε και το αγνοούμε.
* Αντλούμε έγκυρα argmax μέχρι να συγκεντρώσουμε τους K πιο όμοιους χρήστες.
* Υπολογίζουμε τις προβλέψεις με βάση το σύνολο τον πιο όμοιων χρηστών και τις τοποθετούμε στον πίνακα με τα reviews.
* Οι χρόνοι εκτέλεσεις δεν είναι ικανοποιοιτικοί ίσως χρειάζεται να τροποποιήσω τον αλγόριθμο.

In [7]:
import sys
import numpy as np
import time
import copy
from scipy.sparse import csc_matrix,lil_matrix

def user_based(M,K,M_cosine_similarity,removed_cells):
    M_csc = csr_matrix.tocsc(M)
    np.fill_diagonal(M_cosine_similarity,0)
    predictions = []
    predictions_sp = sp_sparse.lil_matrix((len(users),len(business)),dtype=float)
    cell_counter = 0
    start_time = time.time()
    for removed_cell in removed_cells:

        #Με j είναι οι χρήστρες και i είναι οι επιχειρήσεις
        i = removed_cell[0]
        j = removed_cell[1]

        #αριθμός όμοιων επιχειρήσεων που θα συμβάλουν στον υπολογισμό πρόβλεψης της βαθμολογίας που θα έβαζε ο j χρήστης
        k=K

        #Εξάγω τις θέσεις των κελιών όπου άλλοι χρήστες έχουν δώσει βαθμολογία για την j επιχείρηση
        non_zero_ratings = csc_matrix.nonzero(M_csc[:,j])
        number = len(non_zero_ratings[0])

        #Αυτή η μάσκα έχει άσους στις θέσεις όπου τα ratings για την επιχείρηση είναι μη μηδενικά
        mask_data = np.ones((number))
        mask_row = np.zeros((number),dtype=int)
        mask_col = non_zero_ratings[0]
        mask = sp_sparse.csr_matrix((mask_data,(mask_row,mask_col)),shape=(1,len(M_cosine_similarity[i,:])))

        #Εδώ κρατάμε ένα slice με το similarity του χρήστη που μας ενδιαφέρει
        similarity_row = M_cosine_similarity[i,:]
        similarity_row = sp_sparse.csr_matrix(similarity_row)

        similarity_row = similarity_row.toarray()
        mask = mask.toarray()

        result = similarity_row * mask
        #φτιάχνω μια νέα μάσκα στην οποία πια θα κρατήσω μόνο τις K μεγαλύτερες όμοιότητες
        new_mask = lil_matrix((1,len(M_cosine_similarity[i,:])), dtype=float)
        
        #φτιάχνω ένα αντίγραφο του πίνακα με τα συνολικά similarities για να έχω πρόσβαση στις πληροφορίες του πίνακα μετά την τροποποίηση του
        #λόγω της διαδικασίας αναζήτησης των Κ κορυφαίων ομοιοτήτων μέσα στον πίνακα
        copied = copy.deepcopy(result)

        number_of_non_zero_similarities = len(non_zero_ratings[0])
        if k > number_of_non_zero_similarities:
            k = number_of_non_zero_similarities

        while k > 0:
            index = np.argmax(result)
            #τοποθετώ το max similarity στο νέο κενό πίνακα που θα χρησιμοποιήσω για τον υπολογισμό της πρόβλεψης της βαθμολογίας
            new_mask[0,index] = copied.item(0,index)

            #θέτω 0 το max similarity που βρήκα για να πάρω με το argmax την θέση του αμέσως μεγαλύτερου από αυτό
            result[0][index] = 0
            k -= 1

        sum_of_sim = 0
        sum_of_rating = 0
        #υπολογίζω το άθροισμα του παρονομαστή
        sum_of_sim = np.sum(new_mask)

        #υπολογίζω το άθροισμα του αριθμητή
        sum_of_rating = new_mask * M[:,j]
        sum_of_rating = np.sum(sum_of_rating)
        cell_counter += 1

        #καταχωρώ την πρόβλεψη στον πίνακα με τα predictions
        if sum_of_sim != 0:
            prediction = sum_of_rating/sum_of_sim
        else:
            prediction = 0
        predictions_sp[i,j] = prediction
        

    end_time = time.time()
    print(end_time-start_time)

    return predictions_sp

In [ ]:
predictions = user_based(M,20,M_cosine_similarity,removed_cells)
print(predictions)

In [14]:
from math import sqrt
RMSE = sqrt(metrics.mean_squared_error(removed_cells_sp.toarray(), predictions.toarray()))
print(RMSE)

0.09615359433538458


In [15]:
import scipy.sparse.linalg as sp_linalg
import matplotlib.pyplot as plt
k = 10
U,s,V = sp_linalg.svds(M,k,which = 'LM')

print (U.shape, s.shape, V.shape)
print(s)
plt.plot(s[::-1])
plt.ylabel('eigenvalue value')
plt.xlabel('number of eigenvalues')
S = np.diag(s)

(4794, 10) (10,) (10, 4092)
[ 96.85109169  98.9647928  104.66445875 108.89190565 111.56240011
 120.15486432 121.30448493 132.00399943 178.25084078 327.15289469]


In [16]:
print(S)
#data_k = U[:,0:].dot(S[0:,0:]).dot(V[0:,:])
#data_k = U.dot(S).dot(V)
#data_k[data_k>5] = 5
#data_k[data_k<0] = 0

for k in range(k,0,-1):
    reconstructed_M += U[:,k:].dot(S[k:,k:]).dot(V[k:,:])
    error = sqrt(metrics.mean_squared_error(M.toarray(),reconstructed_M))
    print(k)
    print(error)

print(reconstructed_M)

[[ 96.85109169   0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.          98.9647928    0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.         104.66445875   0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.         108.89190565   0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.         111.56240011
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
  120.15486432   0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
    0.         121.30448493   0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
    0.           0.         132.00399943   0